In [ ]:
#결혼자금준비 ( DRA -> MVO )
import pandas as pd
from block_definitions import classify_assets_by_blocks
from mvo import mvo_optimizer
from dra import dynamic_risk_allocation

data_path = '/Users/heunjjang/Desktop/sample.csv'
data = pd.read_csv(data_path)

l1_assets, l2_indices, l3_markets, l4_etfs = classify_assets_by_blocks()

# 사용자 입력
goal_period = int(input("투자 기간 (년): "))
risk_preference = int(input("위험 선호도 (1: 매우 안전, 5: 매우 공격적): "))
risk_aversion = 6 - risk_preference 

final_allocations = {}
upper_weights = {"L1": 1.0}

# L1: DRA
final_allocations["L1"] = dynamic_risk_allocation(data[l1_assets], risk_aversion, goal_period)
# L2 ~ L4: MVO
for level, assets, prev_level in [
    ("L2", l2_indices, "L1"),
    ("L3", l3_markets, "L2"),
    ("L4", l4_etfs, "L3")
]:
    upper_weight = sum(final_allocations[prev_level].values())
    cov_matrix = data[assets].pct_change().dropna().cov().values
    expected_returns = data[assets].pct_change().dropna().mean().values

    weights = mvo_optimizer(
        nodes=assets,
        covariance_matrix=cov_matrix,
        expected_returns=expected_returns,
        weight_bounds=(0.05, 0.7), # 각 자산의 최소/최대 비중
        risk_aversion=risk_aversion
    )

    final_allocations[level] = {asset: weight * upper_weight for asset, weight in zip(assets, weights)}

print("\n### 결혼자금 마련 포트폴리오 ###")
l4_allocation_list = [f"{asset}: {weight * 100:.2f}%" for asset, weight in final_allocations["L4"].items()]
print(l4_allocation_list)



### 결혼자금 마련 포트폴리오 ###
['REET: 5.00%', 'USO: 5.00%', 'MTUM: 10.47%', 'SPLV: 11.75%', 'SRVR: 5.00%', 'XHB: 15.67%', 'SLV: 22.47%', 'JPST: 24.65%']
